# Combination of datasets prior to further modelling

### The purpose of this notebook is combine datasets pulled from different areas of the data warehouse. The initial dataset will be all those currenthly present in the dim person table. Further to this, datasets regarding such areas as disease status, alcohol consumption and social satus will be joined.

In [ ]:
# All packaged required are loaded here
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
pd.options.mode.chained_assignment = None

In [ ]:
df = pd.read_csv('data.txt', sep='\t', header = 0)

In [ ]:
# Here we asssess those who have become deceased in our dataset.
# We create a deceased date from the two existing columns
# We take those who have died in the last 12 months using a cut off
# We then remove those who have died but not in the last 12 months
df_deceased = df[df.IsDeceased == 1]
df_deceased['DeceasedYear'] = df_deceased['DeceasedYear'].astype(int)
df_deceased['DeceasedMonth'] = df_deceased['DeceasedMonth'].astype(int)
df_deceased['DeceasedDate'] = df_deceased['DeceasedYear'].astype(str) + '-' + df_deceased['DeceasedMonth'].astype(str)
df_deceased['DeceasedDate'] = df_deceased['DeceasedDate'].astype(str) + '-01'
df_deceased[["DeceasedDate"]] = df_deceased[["DeceasedDate"]].apply(pd.to_datetime)
df_deceased['Cutoff'] = '2024-07-12'
df_deceased[['Cutoff']] = df_deceased[['Cutoff']].apply(pd.to_datetime)
df_deceased['DeathDifference'] = (df_deceased['Cutoff'] - df_deceased['DeceasedDate']).dt.days
df_deceased['Target'] = df_deceased.apply(lambda row: 1 if row['DeathDifference'] <= 365 else 0, axis=1)
df_deceased = df_deceased[df_deceased.Target == 1]

In [ ]:
# Here we seperate those who are alive
# We then create the same columns in the deceased dataset 
df_alive = df[df.IsDeceased == 0]
df_alive['DeceasedMonth'] = df_alive['DeceasedMonth'].fillna(1.0).astype(int)
df_alive['DeceasedYear'] = df_alive['DeceasedYear'].fillna(1900).astype(int)
df_alive['DeceasedDate'] = df_alive['DeceasedYear'].astype(str) + '-' + df_alive['DeceasedMonth'].astype(str)
df_alive['DeceasedDate'] = df_alive['DeceasedDate'].astype(str) + '-01'
df_alive[["DeceasedDate"]] = df_alive[["DeceasedDate"]].apply(pd.to_datetime)
df_alive['Cutoff'] = '2024-07-12'
df_alive[['Cutoff']] = df_alive[['Cutoff']].apply(pd.to_datetime)
df_alive['DeathDifference'] = (df_alive['Cutoff'] - df_deceased['DeceasedDate']).dt.days
df_alive['Target'] = 0


In [ ]:
# Here we join the deceased and alive dataset
# We then shuffle the dataset to introduce some randomness
df = df_deceased._append(df_alive)
df = shuffle(df)

In [ ]:
# We remove any columns regarded as unecessary in order to streamline the dataset.
df = df.drop(['Pseudonym','CurrentGpSurgeryKey','CurrentGpSurgeryRegistrationDateKey','Gender','OutputArea',
             'LowerSuperOutputArea','MiddleSuperOutputArea','MiddleSuperOutputAreaName','Latitude','Longitude',
             'WardCode','WardName','LocalAuthorityCode','FiveYearAgeBand','FiveYearAgeBandOrder',
             'TenYearAgeBandOrder','SegmentAgeBand','SegmentAgeBandOrder','EthnicityReadCode',
             'EthnicityCodeDescription','EthnicityGroupDescription','EthnicityRecordedDateKey',
             'Cutoff','LocalAuthorityName','SpeaksEnglish','IsDeceased','DeceasedYear','DeathDifference','TenYearAgeBand',
             'EthnicityGroupDetailDescription','GenderShort'],axis = 1)

In [ ]:
# We remove any rows which have NAs for future compuational reasons
df = df.dropna()

,PersonKey,Age,BirthMonth,IsLsoaCoastal,ImdDecile,LowerSuperOutputAreaDeprivationDecile,LowerSuperOutputAreaIncomeDecile,LowerSuperOutputAreaEmploymentDecile,LowerSuperOutputAreaEducationSkillsAndTrainingDecile,LowerSuperOutputAreaHealthDeprivationAndDisabilityDecile,...,MosaicKeyFeature2,MosaicKeyFeature3,MosaicKeyFeature4,MosaicKeyFeature5,MosaicKeyFeature6,DeceasedMonth,IsOptedOut,IsNewDorsetRegistration,DeceasedDate,Target
852360,288892,16,3.0,1,5.0,13414.0,5.0,4.0,3.0,3.0,...,Families with children,Oil/solid fuel central heating,Internet via smartphone,Single trip travel insurance,Free mobile phone apps,1,0,0,1900-01-01,0
270639,776054,48,8.0,0,10.0,31005.0,10.0,9.0,9.0,9.0,...,No children,Rented 1 bed flats,Work full-time,Text messages,Electric central heating,1,0,0,1900-01-01,0
77598,1357606,17,7.0,0,5.0,16056.0,6.0,5.0,2.0,7.0,...,"Older households, no children",Homeowners,Garden or allotment,Internet from desktop/laptop,Free mobile phone apps,1,0,0,1900-01-01,0
873012,296125,29,7.0,0,5.0,13189.0,4.0,4.0,3.0,5.0,...,No children,Rented 1 bed flats,Work full-time,Text messages,Electric central heating,1,0,0,1900-01-01,0
379498,814124,16,5.0,1,3.0,7415.0,2.0,3.0,2.0,3.0,...,School-age children,No qualifications,Low discretionary income,Standard current/savings account,Single trip travel insurance,1,0,0,1900-01-01,0


In [ ]:
# Here we can now see how many people we habe who died within the last 12 months
df.Target.value_counts()

Target
0    801013
1      8710
Name: count, dtype: int64

In [ ]:
# All the mosaic features are categorical examples. These will need to be turned into numeric features for modelling.
# A for loop is used to identify those columns which are mosaic and turn them intp numbers
for col in df.columns:
    if 'Mosaic' in col:
        df[col] = pd.factorize(df[col])[0]

## The joining of disease condition status

In [ ]:
## Here we import data for various diseases
disease = pd.read_csv('disease.csv')

In [ ]:
# This piece of code determines whether or not a disease was diagnosed within the last 5 years.
# We pick a reference date to calculate the 5 year period
reference_date = datetime(2024, 7, 12)

# This is a function which determines if a date is wiithin 5 years of our reference
def within_5_years(date, reference):
    return int(abs((date - reference).days) <= 5 * 365)

# Loop through columns with 'date' in their names and process them
for col in disease.columns:
    if 'Date' in col:
        try:
            # Attempt to convert the column to datetime
            disease[col] = pd.to_datetime(disease[col], errors='coerce')
            
            # If conversion is successful, create the new binary column
            new_col_name = col[0:5] + '_within_5_years'
            disease[new_col_name] = disease[col].apply(lambda x: within_5_years(x, reference_date) if pd.notnull(x) else 0)
        except Exception as e:
            # Handle any unexpected exceptions
            print(f"An error occurred while processing column {col}: {e}")

In [ ]:
# We join the diseases data onto our intial data based on the personkey
df = df.merge(disease, on='PersonKey', how='left')

In [ ]:
# We replace any missing data with 0s as we can assume a person does not have the disease if not in the table
df = df.replace(np.nan, 0)
df.Target.value_counts()

Target
0    801013
1      8710
Name: count, dtype: int64

## We now join data concerning patients visits to the GP

In [ ]:
# We read in the data and add the required the column names.
# We also add a one to each entry, this is so we can sum the number of appointments later.
# We then group by the number of appointments for each person and reset the index
Primary = pd.read_csv('Primary.txt', sep='\t', header = 0)
prim_cols = ['PersonKey','AppointmentDateKey','DidNotAttend']
Primary.columns = prim_cols
Primary['Appointment'] = 1
app_group = Primary.groupby('PersonKey')['Appointment'].sum()
primary_app = app_group.reset_index()

In [ ]:
# We again merge the new GP data onto the evolving data frame
df = df.merge(primary_app, on='PersonKey', how='left')
df.Appointment.isna().sum()

61824

In [ ]:
# We now convert the appointdate of each persons appointments to datetime and extract there years
Primary['AppointmentDateKey'] = pd.to_datetime(Primary['AppointmentDateKey'])
Primary['Year'] = Primary.AppointmentDateKey.dt.year

In [ ]:
# We now createb a pivot table, which sums the number of apps a person has per year
# We then reset the index and take only those columns for 2023 and 2024
# We reset the column names and join this data to our dataframe
pivoted = Primary.pivot_table(index='PersonKey', columns='Year', values='Appointment',aggfunc='sum')
pivoted = pivoted.reset_index()
pivoted = pivoted[['PersonKey',2023,2024]]
pivoted = pivoted.reset_index(drop=True)
cols = ['PersonKey','2023_appointments','2024_appointments']
pivoted.columns = cols
df = df.merge(pivoted, on = 'PersonKey', how = 'left')

## Here we join any data concerning a persons trips to A&E

In [ ]:
# Here we read in our emergency data and apply the relevant column names.
# We now again apply a count and groupby a persons number of visits.
# We then reset our index and finally merge the datasets
Emergency = pd.read_csv('Emergency.txt', sep='\t', header = None)
Emergency.columns = ["PersonKey", "AppointmentDateKey", "DidNotAttend"]
Emergency['Visit'] = 1
em_group = Emergency.groupby('PersonKey')['Visit'].sum()
emergency_app = em_group.reset_index()
df = df.merge(emergency_app, on='PersonKey', how='left')

In [ ]:
# The date of each A&E visit is then converted to a datetime and the years extracted.
# Again a pivot table sums the number of visits for each person.
# We then select only 2023,2024 visits, apply new column names and merge onto our dataset.
# We can replace NAs with 0s again the person had no visits if not in the dataset.
Emergency['AppointmentDateKey'] = pd.to_datetime(Emergency['AppointmentDateKey'])
Emergency['Year'] = Emergency.AppointmentDateKey.dt.year
pivoted = Emergency.pivot_table(index='PersonKey', columns='Year', values='Visit',aggfunc='sum')
pivoted = pivoted.reset_index()
pivoted = pivoted[['PersonKey',2023,2024]]
pivoted = pivoted.reset_index(drop=True)
cols = ['PersonKey','2023_visitsA_E','2024_visitsA_E']
pivoted.columns = cols
df = df.merge(pivoted, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## We now add in further data regarding a person's alcohol consumption

In [ ]:
# The alcohol data is read in and the appropiate columns given
# The data is then merged onto our dataset and nas are dealt with
alcohol = pd.read_csv('Alcohol.txt', sep='\t', header = None)
alc_cols = ['PersonKey','AlcoholConsumptionUnitsPerWeek','AlcoholConsumptionUnitsPerWeekDateKey']
alcohol.columns = alc_cols
df = df.merge(alcohol, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## Further clinical data about each patient is then appended here

In [ ]:
# The data is read in and columns added.
# We then append the new data and fill in any missing values
Clinical = pd.read_csv('clinical.txt', sep='\t', header = None)
clinical_cols = ['PersonKey','BmiValue','BmiCheckDateKey','IsObese','ObeseCheckDateKey','CholesterolValue',
                 'CholesterolCheckDateKey','FluVaccinationRecorded','FluVaccinationDatekey','SeizuresPerYear',
                 'SeizuresPerYearCheckDateKey','ReducedMobility','ReducedMobilityDateKey','ConfinedToChair',
                 'ConfinedToChairDateKey','BedRidden','BedRiddenDateKey','HeartRateValue','HeartRateValueCheckDateKey',
                'SystolicBpValue','DiastolicBpValue','MalnutritionUniversalScreeningTool','MalnutritionUniversalScreeningToolScoreValue']
Clinical.columns = clinical_cols
df = df.merge(Clinical, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## Here we attach any data referring to mental health services

In [ ]:
# We read in the datat for mental health referrals
M_health = pd.read_csv('Mental.csv')
M_health['M_visit'] = 1

In [ ]:
# We name and apply the relevant column names
# We then add a numeric value called visit so we can sum later
# We group by the person key and number of referrals
# We reset the index and then merge on the personkey column
M_het = M_health[['PersonKey','ReferralID','M_visit']]
M_group = M_het.groupby('PersonKey')['M_visit'].sum()
M_app = M_group.reset_index()
df = df.merge(M_app, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

In [ ]:
# Here we create a pivot table for the reasons for mental health referral
# We select only a few reasons to be appended to the dataset
pivoted = M_health.pivot_table(index='PersonKey', columns='ReferralReason', values='M_visit',aggfunc='sum')
pivoted = pivoted.reset_index()
pivoted = pivoted[['PersonKey','Anxiety','Depression','Self harm behaviours','Eating disorders','Bi polar disorder',
                   'Drug and alcohol difficulties','Ongoing or Recurrent Psychosis']]
pivoted = pivoted.reset_index(drop=True)
df = df.merge(pivoted, on='PersonKey', how='left')
df = df.replace(np.nan, 0)


In [ ]:
# Here we group on the number of referrals per person
# We then total those for 2023 and 2024 and append them to a dataframe.
M_health['ReferralDate'] = pd.to_datetime(M_health['ReferralDate'])
M_health['Year'] = M_health.ReferralDate.dt.year
pivoted = M_health.pivot_table(index='PersonKey', columns='Year', values='M_visit',aggfunc='sum')
pivoted = pivoted.reset_index()
pivoted = pivoted[['PersonKey',2023,2024]]
pivoted = pivoted.reset_index(drop=True)
cols = ['PersonKey','2023_visits_M','2024_visits_M']
pivoted.columns = cols
df = df.merge(pivoted, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

C:\Users\rowt\AppData\Local\Temp\ipykernel_22944\396056291.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  M_health['ReferralDate'] = pd.to_datetime(M_health['ReferralDate'])


In [ ]:
# We read in the data for the number of mental health appointments the person has
M_contact = pd.read_csv('mental_contact.txt', sep='\t', header = None)

In [ ]:
# We create our columns and apply them
# We apply a numeric column for later summation
# We group by the personkey and number of appointments
# Finally we merge on the personkey and replace nans.
Meds_contact_cols = ['PersonKey','CareContactDateKey']
M_contact.columns = Meds_contact_cols
M_contact['MH_app'] = 1
M_apps = M_contact.groupby('PersonKey')['MH_app'].sum()
M_apps = M_apps.reset_index()
df = df.merge(M_apps, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## Here we append data based on waiting lists

In [ ]:
# We read in the data for the number of waiting lists a person is on
Waiting = pd.read_csv('Waiting.txt', sep='\t', header = None)

C:\Users\rowt\AppData\Local\Temp\ipykernel_22944\291117941.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  Waiting = pd.read_csv('Waiting.txt', sep='\t', header = None)


In [ ]:
# We create our columns and apply them
# We apply a numeric column for later summation
# We group by the personkey and number of waiting lists
# Finally we merge on the personkey and replace nans.
Waiting_cols = ['PersonKey','Date','Waiting_time']
Waiting.columns = Waiting_cols
Waiting['Lists'] = 1
Waiting = Waiting[['PersonKey','Lists']]
waiting_l = Waiting.groupby('PersonKey')['Lists'].sum()
Waiting_l = waiting_l.reset_index()
df = df.merge(waiting_l, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## Here we add some additional information regarding social status

In [ ]:
# We read in the data for the number of waiting lists a person is on
Social = pd.read_csv('social.csv')

In [ ]:
# We merge on the personkey and replace nans.
df = df.merge(Social, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## Here we add some data regarding fractures and falls

In [ ]:
# We read in the data for the number of falls and fractures a person has
fractures =  pd.read_csv('fractures.txt', sep='\t', header = None)

In [ ]:
# We create our columns and apply them
# We group by the personkey and number falls etc
# Finally we merge on the personkey and replace nans.
frac_cols = ['PersonKey','RespAdmission','Fractures','Falls']
fractures.columns = frac_cols
total_fracs = fractures.groupby('PersonKey')[['RespAdmission','Fractures','Falls']].sum()
total_fracs = total_fracs.reset_index()
df = df.merge(total_fracs, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## Here we add information regarding a person's smoking status

In [ ]:
# We read in smoking data for our population
Smoking = pd.read_csv('Smoking.txt', sep='\t', header = None)

In [ ]:
# We apply our column names
# We then use a function to assign a binary smoking status based on values
# These are smoker and non-smoker. 
# The smokingstatus variable was then dropped
Smoking_cols = ['PersonKey','SmokingStatus']
Smoking.columns = Smoking_cols
def set_value(row_number, assigned_value):
    return assigned_value[row_number]
event_dictionary = {'Smoker': 1, 'Ex-Smoker': 1, 'Never Smoked':0,'UNKNOWN':0}
# Add a new column named 'Price'
Smoking['Smoked'] = Smoking['SmokingStatus'].apply(set_value, args=(event_dictionary, ))
Smoking = Smoking.drop(['SmokingStatus'],axis = 1)
df = df.merge(Smoking, on='PersonKey', how='left')
df = df.replace(np.nan, 0)


## We now add information regarding disabilities

In [ ]:
# We read in data regarding disabilities
# We then assign a value of 1 for each condition
# We groupby the number of conditions per person
# We then append the data to our dataframe
disable = pd.read_csv('Disability.csv')
disable['D_condition'] = 1
total_disability = disable.groupby('PersonKey')[['D_condition']].sum()
total_disability = total_disability.reset_index()
df = df.merge(total_disability, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## Here we add some further information regarding Adults social services

In [ ]:
# We read in data regarding adult social services
# We remove duplicates on the personkey
# We turn the categorical data into numerical and add 1 in order to fill nans later
Adult = pd.read_csv('Adult.csv')
Adult = Adult.drop_duplicates(subset=['PersonKey'],keep = 'first')
Adult['PrimarySupportReason'] = pd.factorize(Adult['PrimarySupportReason'])[0]
Adult['PrimarySupportReason'] = Adult['PrimarySupportReason'] + 1
df = df.merge(Adult, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## Now we can bring in data regarding 111 calls

In [ ]:
# We read in data regarding 111 calls
# We apply column names and drop duplicates on the caseid
# We then groupby on the number of calls and append to our data
One_emergency = pd.read_csv('one_urgent.txt', sep='\t', header = None)
One_emergency_cols = ['PersonKey','Caseid','CaseStartDateTime']
One_emergency.columns = One_emergency_cols
One_emergency = One_emergency.drop_duplicates(subset=['Caseid'], keep='first')
One_emergency['Incidents'] = 1
One_emergency = One_emergency.groupby('PersonKey')[['Incidents']].sum()
One_emergency = One_emergency.reset_index()
df = df.merge(One_emergency, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

## We can now add some information regarding Ambulance activity per person

In [ ]:
# We read in data regarding ambulance usage
# We add our column names and drop duplicates on incidentnumber
Ambulance = pd.read_csv('Ambulance.txt', sep='\t', header = None)
Ambulance_cols = ['PersonKey','IncidentNumber','IncidentDateKey']
Ambulance.columns = Ambulance_cols
Ambulance = Ambulance.drop_duplicates(subset=['IncidentNumber'], keep='first')
Ambulance['Call'] = 1
Ambulance = Ambulance.groupby('PersonKey')[['Call']].sum()
Ambulance = Ambulance.reset_index()
df = df.merge(Ambulance, on='PersonKey', how='left')
df = df.replace(np.nan, 0)


## Here we read in data regarding covid-19

In [ ]:
Covid_vac = pd.read_csv('Covid_vac.csv')

In [ ]:
# Here we append the covid vulnerability status of each person
Covid_risk = Covid_vac.iloc[:,0:4]
df = df.merge(Covid_risk, on='PersonKey', how='left')
df = df.replace(np.nan, 0)


In [ ]:
# Here we append the number of covid vaccines each person has had
# We sum the number of vaccines into a total column
column_names = ['CovidVaccinationPart1','CovidVaccinationPart2', 'CovidVaccinationBooster']
Covid_vac['Total_Covid_Vaccines']= Covid_vac[column_names].sum(axis=1)
Total_vac = Covid_vac[['PersonKey','Total_Covid_Vaccines']]
df = df.merge(Total_vac, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

In [ ]:
# We now add whether a person declined their vaccine or had a poor reaction
Covid_declined = Covid_vac[['PersonKey','CovidVaccineDeclined','CovidVaccineAdverseReaction']]
df = df.merge(Covid_declined, on='PersonKey', how='left')
df = df.replace(np.nan, 0)

In [ ]:
# Here we drop any columns with dates in them
def drop(df):
    for col in df.columns:
        if 'Date' in col:
            df = df.drop([col], axis =1)
        else:
            print('no drop')
        
    return df

In [ ]:
df = drop(df)

no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop
no drop


In [ ]:
df.to_csv('dataframe.txt')